<a href="https://colab.research.google.com/github/erikrozi/acmlab/blob/main/unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in the Shared Google Drive
FOLDERNAME = 'Shared drives/Unsupervised'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/{}'.format(FOLDERNAME))

%cd /content/drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/Shared drives/Unsupervised


In [2]:
# Importing the standard ML libraries...
%load_ext autoreload
%autoreload 2

import pandas as pd                     # to process our data
import matplotlib.pyplot as plt         # graphing
import numpy as np                      # matrices

import torch
import torchvision                      # for MNIST dataset/working with images

# take advantage of GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load Census Data

In [3]:
tax_returns_data = pd.read_csv("16zpallnoagi.csv")
cleaned_tax_data = pd.DataFrame(tax_returns_data, columns = ["ZIPCODE", "N1", "A02650"])

cleaned_tax_data

,ZIPCODE,N1,A02650
0,0,2016530,115806207
1,35004,5210,290260
2,35005,3100,121690
3,35006,1200,57418
4,35007,11940,697882
...,...,...,...
29969,83126,170,10687
29970,83127,1470,109088
29971,83128,920,88463
29972,83414,200,44409


In [4]:
ziplatlon_data = pd.read_csv("ziplatlon.csv")
ziplatlon_data

,zip;city;state;latitude;longitude;timezone;dst;geopoint
71937;Cove;AR;34.398483;-94.39398;-6;1;34.398483,-94.39398
72044;Edgemont;AR;35.624351;-92.16056;-6;1;35.624351,-92.16056
56171;Sherburn;MN;43.660847;-94.74357;-6;1;43.660847,-94.74357
49430;Lamont;MI;43.010337;-85.89754;-5;1;43.010337,-85.89754
52585;Richland;IA;41.194129;-91.98027;-6;1;41.194129,-91.98027
...,...
66860;Madison;KS;38.126728;-96.16635;-6;1;38.126728,-96.16635
59602;Helena;MT;46.696501;-111.96895;-7;1;46.696501,-111.96895
16063;Zelienople;PA;40.783001;-80.13343;-5;1;40.783001,-80.13343
18913;Carversville;PA;40.376499;-75.04129;-5;1;40.376499,-75.04129


# Load the imagery data


In [ ]:
zip_path = "?"
!cp "{zip_path}" .
!unzip *.zip
!rm *.zip

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class TileImageryDataset(Dataset):
  """Tile imagery dataset."""

  def __init__(self, images_zip_path):
    # Load into tensors

    # Perform data augmentation

# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
  def load_images():
    tensors = []
    for x in range(2794, 2839):
      for y in range(6528, 6572):
        image = Image.open(f"images/14_{x}_{y}.jpg").convert("RGB")
        # TODO change
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        input_tensor = preprocess(image)
        # tensory push thing?